In [1]:
# Importação das bibliotecas necessárias para acessar o serviço Groq
# e carregar o modelo de linguagem Llama 3.3 Versatile.
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

# Carregar as variáveis de ambiente do arquivo .env
load_dotenv()

# Obter a chave da API do Groq a partir das variáveis de ambiente
# Verificar se a chave da API foi definida corretamente
api_key = os.getenv('GROG_API_KEY')
if not api_key:
    raise ValueError("GROG_API_KEY não está definida. Por favor, defina a chave da API no arquivo .env.")

# Inicializar o modelo de linguagem Llama 3.3 Versatile usando a chave da API
llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    api_key=api_key
)

In [2]:
# Exemplo de uso do modelo de linguagem para invocar uma pergunta
prompt = "Qual é a capital da França?"

# Invocar o modelo de linguagem com o prompt
response = llm.invoke(prompt)

# Exibir a resposta do modelo
print(f"Resposta do modelo: {response}")

Resposta do modelo: content='A capital da França é Paris.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 43, 'total_tokens': 52, 'completion_time': 0.001920244, 'prompt_time': 0.017961858, 'queue_time': 0.285294487, 'total_time': 0.019882102}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--899bb3d6-1b7d-457d-b08d-fa7bc641cf45-0' usage_metadata={'input_tokens': 43, 'output_tokens': 9, 'total_tokens': 52}


### Com dados de fontes internas

In [3]:
# URL do documento PDF que será utilizado como fonte de dados
url = 'https://raw.githubusercontent.com/allanspadini/curso-flash-rag/main/m2m_strategy_and_objectives_development.pdf'


In [4]:
# Importação do carregador de documentos PDF da biblioteca LangChain Community
from langchain_community.document_loaders import PyPDFLoader

# Carregar o documento PDF a partir da URL
# Criar uma variável chamada `loader` que utiliza o PyPDFLoader para carregar o PDF
loader = PyPDFLoader(url)

# Carregar o documento PDF de forma que tenha vários blocos, ou seja, conteúdo por página
pages = []

# Utilizar o método lazy_load do loader para carregar as páginas do PDF
for page in loader.lazy_load():
    pages.append(page)

In [5]:
# Exibir os metadados
print(f"{pages[0].metadata}\n")

# Explorar o conteúdo do primeiro bloco carregado
print(f"Conteúdo da primeira página: {pages[0].page_content}\n")

{'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 18.2 (Windows)', 'creationdate': '2023-04-06T17:28:28-04:00', 'moddate': '2023-04-06T17:29:25-04:00', 'trapped': '/False', 'source': 'https://raw.githubusercontent.com/allanspadini/curso-flash-rag/main/m2m_strategy_and_objectives_development.pdf', 'total_pages': 78, 'page': 0, 'page_label': '1'}

Conteúdo da primeira página: NASA’S 
MOON TO MARS 
STRATEGY AND 
OBJECTIVES 
DEVELOPMENT
A blueprint for sustained 
human presence and 
exploration throughout 
the Solar System
National Aeronautics and
Space Administration



In [6]:
# Criação de uma base de dados vetorial para armazenar as informações do PDF, usando langchain
# Importação do InMemoryVectorStore da biblioteca LangChain Core, que permite armazenar vetores em memória
from langchain_core.vectorstores import InMemoryVectorStore

# Usar o HuggingFaceEmbeddings para gerar embeddings dos textos, ou seja, tranformar o texto em vetores
from langchain_huggingface import HuggingFaceEmbeddings

# Modelo de embeddings do HuggingFace para transformar o texto em vetores, modelo aberto e gratuito
embed_model = HuggingFaceEmbeddings(model_name="mixedbread-ai/mxbai-embed-large-v1")
# necessário instalar pip install sentence-transformers para usar HuggingFaceEmbeddings


# Criar uma base de dados vetorial em memória a partir dos documentos carregados
vector_store = InMemoryVectorStore.from_documents(pages, embed_model)



d:\Python\RAG_e_Agentes_de_IA\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Python\RAG_e_Agentes_de_IA\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Jefferson\.cache\huggingface\hub\models--mixedbread-ai--mxbai-embed-large-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order

In [8]:
# Associando a variável ao VectorStore
docs = vector_store

# Realizar uma busca de similaridade no VectorStore para encontrar documentos relevantes
docs = vector_store.similarity_search

# Realizando a busca por similaridade
#docs = vector_store.similarity_search()

# Exemplo de busca por similaridade (PDF em inglês, então a consulta também deve ser em inglês)
docs = vector_store.similarity_search("Objectives Development Process")

# Mostrar os resultados da busca com k=2 para limitar o número de resultados
docs = vector_store.similarity_search("Objectives Development Process", k=2)

# Exibir os resultados da busca
for doc in docs:
    print(f"Conteúdo: {doc.page_content}\n")
    print(f"Metadados: {doc.metadata}\n")


Conteúdo: APPENDIX B: OBJECTIVES DEVELOPMENT PROCESS
In accordance with the methodology principle of Architect from the Right, in November 2021, the NASA Executive Council 
tasked the mission directorates to draft a set of top-level goals and objectives necessary to achieve the Moon to Mars 
endeavor. The task would use Federated Board processes to reach consensus. 
The top-level goals and objectives were binned into four distinct tracks:
• Science
• Infrastructure
• Transportation and Habitation
• Operations
Each of the mission directorate leaders on the Federated Board led the development of corresponding objectives: Science was 
led by the Science Mission Directorate; Infrastructure was led by the Space Technology Mission Directorate; Transportation 
and Habitation was led by the Exploration Systems Development Mission Directorate; and Operations was led by the Space 
Operations Mission Directorate. The Aeronautics Mission Directorate supported the Science Mission Directorate with t